In [23]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pipeline
import tflearn
import tensorflow as tf
from sklearn.metrics import accuracy_score
import numpy as np

import datetime

from utils import *
from data_utils import *

sys.path.append("../") # so we can import models.
from models import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:

def _load_model(model_id, network, n_classes=10, load_checkpoint=False, is_training=False):
    # should be used for all models
    print ('Loading model...')

    tensorboard_dir = '../tensorboard_logs/' + model_id + '/'
    checkpoint_path = '../checkpoints/' + model_id + '/'
    best_checkpoint_path = '../best_checkpoints/' + model_id + '/'

    print (tensorboard_dir)
    print (checkpoint_path)
    print (best_checkpoint_path)

    check_if_path_exists_or_create(tensorboard_dir)
    check_if_path_exists_or_create(checkpoint_path)
    check_if_path_exists_or_create(best_checkpoint_path)

    if is_training:
        model = tflearn.DNN(network, tensorboard_verbose=2, tensorboard_dir=tensorboard_dir,
                            checkpoint_path=checkpoint_path, best_checkpoint_path=best_checkpoint_path, max_checkpoints=3)
    else:
        model = tflearn.DNN(network)

    if load_checkpoint:
        checkpoint = tf.train.latest_checkpoint(checkpoint_path)  # can be none of no checkpoint exists
        if checkpoint and os.path.isfile(checkpoint):
            # model.load(checkpoint, weights_only=True, verbose=True)
            model.load(checkpoint, verbose=True)
            print ('Checkpoint loaded.')
        else:
            print ('No checkpoint found. ')

    print ('Model loaded.')
    return model

In [6]:
networks = simple_cnn.build_network([20, 100])

In [15]:
coarseModel = networks[0]
fineModel = networks[1]

In [17]:
X, Y_coarse, X_test, Y_test_coarse = pipeline.load_data(dataset="cifar100_coarse")
X, Y_fine, X_test, Y_test_fine = pipeline.load_data(dataset="cifar100_fine")

Attempting to load dataset cifar100_coarse ...
loading cifar batch training batch 1 of 1
loading cifar batch testing batch 1 of 1
Attempting to load dataset cifar100_fine ...
loading cifar batch training batch 1 of 1
loading cifar batch testing batch 1 of 1


In [18]:
model_id_coarse = "coarse_simple_cnn"
model_id_fine = "fine_simple_cnn"

In [27]:
graph_to_use = tf.Graph()
with graph_to_use.as_default():
    coarse_model = _load_model(model_id_coarse, coarseModel, n_classes=20, load_checkpoint=False, is_training=True)
    date_time_string = datetime.datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
    run_id = "{}_{}".format(model_id_coarse, date_time_string)
    # Train using classifier
    coarse_model.fit(X, Y_coarse, n_epoch=50, shuffle=True, validation_set=(X_test, Y_test_coarse),
              show_metric=True, batch_size=96, run_id=run_id, snapshot_step=100)

Loading model...
../tensorboard_logs/coarse_simple_cnn/
../checkpoints/coarse_simple_cnn/
../best_checkpoints/coarse_simple_cnn/


ValueError: No variables to save